#### In this notebook, assuming an effective size of 16 species, we will look at CS (BPDN,ADMM) on effectively 186 points. For each species, we will only look at predictions on the points where the species was present to begin with. (this is the second set of species that maximises data points)

#### import the relevant files and modules

In [42]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
plt.style.use('ggplot')
plt.style.use('seaborn-dark-palette')
#iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
#B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")


import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_percentage_error as mape

In [2]:
#X = pd.read_pickle("~/bge-analysis-simv3/X16.pkl")

In [3]:
import matplotlib as mpl
from matplotlib import pyplot
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'sans-serif'
def set_violin_color(vp,color,mediancolor):
    plt.setp(vp['bodies'], facecolor=color, edgecolor="white",alpha=0.7)
    plt.setp(vp['cbars'], color = color)
    plt.setp(vp['cmins'], color = "black")
    plt.setp(vp['cmaxes'], color = "black")
    plt.setp(vp['cmeans'], color = "black")
    plt.setp(vp['cmedians'],color = mediancolor)

def make_violin_plot(dataframe,col0,mediancolor,legendlabel,diff):
    stepepi_master0 = dataframe
    bp0 = plt.violinplot(stepepi_master0, positions=np.array(range(len(stepepi_master0)))*3.0+diff,showmeans = True,showmedians=True)
    positions=np.array(range(len(stepepi_master0)))
    set_violin_color(bp0,col0,mediancolor)
    plt.plot([], c=col0, label=legendlabel)
    plt.legend(loc='upper left') 
    return positions

import seaborn as sns
colorlist1 = sns.color_palette("bright").as_hex()
sns.color_palette("bright")

from sklearn.metrics import r2_score

In [4]:
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

#### read the effective species presence-absence

In [7]:
pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

#### read the steady states (relative abundances)

In [10]:
steadystate = pd.read_pickle("~/compressed_sensingv1/realdatasets/ophelli-ryan/eff-16-list1-red-sst-REL.pkl")
sst = steadystate.T

labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([0.5,0.6,1.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()
lamf = lamf[:7]
lamf

samp = 100
n = 2**16
cv_splitsl = [2,3,5,6,7]
random_state = None
laml = lamf

err = [] 
errlin  = [] 
for cv_splits in [2,3,5,6,7]:
    for species in range(6,8):
            m = int(samp*(ones_len[species])/100)
            cvdata = m
            ri = random.sample(list(species_ones[species]),m)
            ri.sort() 
            startt = time.time()
            y2 = sst[ri].T[species]
            data_present = y2[ri].T.index
            xs = list(data_present)
            random.shuffle(xs)
            data_present = np.array(xs)
            kf = KFold(n_splits=cv_splits)
            kf.get_n_splits(data_present)
            KFold(n_splits=cv_splits, random_state=random_state, shuffle=False)
            for lmda in laml:
                for train_index, test_index in kf.split(data_present):
                    X_train, X_test = data_present[train_index], data_present[test_index]
                    rinew = list(X_train)
                    rileft = list(X_test)
                    D = iHiV[rinew,:]
                    y3 = y2[rinew]
                    s2 = np.array([y3.values])
                    s3 = s2.T
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(iHiV,x.ravel())
                    err.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])
                    D = X[rinew,:]
                    y3 = y2[rinew]
                    s2 = np.array([y3.values])
                    s3 = s2.T
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(X,x.ravel())
                    errlin.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits,len(rileft),len(rinew)])

errdf  = pd.DataFrame(err)
errdflin = pd.DataFrame(errlin)

pd.to_pickle(errdf,"16l1-68-bge.pkl")

pd.to_pickle(errdflin,"16l1-68-lin.pkl")

In [75]:
errdf = pd.read_pickle("16l1-1012-bge.pkl")
errdflin = pd.read_pickle("16l1-1012-lin.pkl")
errdf1 = pd.read_pickle("16l1-68-bge.pkl")
pd.concat([errdf,errdf1])

,0,1,2,3,4,5,6,7
0,0.00001,10,"[0.35180006494642324, 0.5585349009279805, 0.52...","[173, 165, 1440, 178, 33, 16416, 545, 16545, 8...","[161, 39, 8224, 167, 40, 162, 38, 1187, 2080, ...",2,33,33
1,0.00001,10,"[0.5354116436779114, 0.5972184592844294, 0.605...","[161, 39, 8224, 167, 40, 162, 38, 1187, 2080, ...","[173, 165, 1440, 178, 33, 16416, 545, 16545, 8...",2,33,33
2,0.00028,10,"[0.35212798156452363, 0.5604671761291942, 0.52...","[173, 165, 1440, 178, 33, 16416, 545, 16545, 8...","[161, 39, 8224, 167, 40, 162, 38, 1187, 2080, ...",2,33,33
3,0.00028,10,"[0.5597458192221192, 0.6021598710552905, 0.583...","[161, 39, 8224, 167, 40, 162, 38, 1187, 2080, ...","[173, 165, 1440, 178, 33, 16416, 545, 16545, 8...",2,33,33
4,0.00794,10,"[0.35049416478137735, 0.5399289032699931, 0.58...","[173, 165, 1440, 178, 33, 16416, 545, 16545, 8...","[161, 39, 8224, 167, 40, 162, 38, 1187, 2080, ...",2,33,33
...,...,...,...,...,...,...,...,...
317,1.00000,7,"[0.2971533408143746, 0.34885621311341264, 0.34...","[418, 260, 49412]","[33028, 258, 256, 257, 289, 416, 1312, 417, 29...",7,3,15
318,1.00000,7,"[0.32856116645232386, 0.21617097573657895, 0.3...","[1312, 417, 294]","[33028, 258, 256, 257, 289, 416, 418, 260, 494...",7,3,15
319,1.00000,7,"[0.27604998016474724, 0.25395480688185545]","[384, 2465]","[33028, 258, 256, 257, 289, 416, 418, 260, 494...",7,2,16
320,1.00000,7,"[0.4130282164334828, 0.2350368368648507]","[25345, 288]","[33028, 258, 256, 257, 289, 416, 418, 260, 494...",7,2,16


In [67]:
crossdf = [] 
for m in cv_splitsl:
    samp = errdf[errdf[5]==m]
    for speciesc in range(10,12):
        species = speciesc
        sp1 = samp[samp[1]==speciesc]
        for rholamb in laml:
            rho1 = sp1[sp1[0]==rholamb]
            r2 = [] 
            mape2 = [] 
            yallbest  = []
            yallact  = [] 
            for iterii in range(m):
                idx = rho1[3].iloc[iterii]
                yact = sst[idx].T[species].values
                ybest = rho1[2].iloc[iterii]
                ybestdf1 = pd.DataFrame(ybest)
                ybestdf1[ybestdf1[0]<1e-3]=0
                slope, intercept,r_value, p_value, std_err = stats.linregress(ybestdf1[0].values,yact)
                r2.append(r2_score(yact,ybestdf1[0].values))
                mape2.append(mape(yact,ybestdf1[0].values))
                yallbest.append(ybestdf1[0].values)
                yallact.append(yact)
            r2df = pd.DataFrame(r2)
            r2bf = pd.DataFrame()
            mape2df = pd.DataFrame(mape2)
            if m == 2:
                ybestagg = [*yallbest[0],*yallbest[1]]
                yactagg = [*yallact[0],*yallact[1]]
            if m == 3:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2]]
            if m == 5:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
            if m == 6:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
            if m == 7:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
            if m == 9:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
            slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
            #print(slope, intercept,r_value**2)
            #r2df = pd.DataFrame(r2)
            crossdf.append([speciesc,r2_score(yactagg,ybestagg),mape(yactagg,ybestagg),slope,intercept,r_value**2,rholamb,m,r2df[0].mean(),mape2df[0].mean()])

In [68]:
crossval = pd.DataFrame(crossdf)

In [69]:
crossval[crossval[7]==3]

,0,1,2,3,4,5,6,7,8,9
14,10,0.350651,2.544219e-01,0.765528,0.151685,0.497596,0.00001,3,0.282423,2.544219e-01
15,10,0.364176,2.514736e-01,0.769199,0.149090,0.504949,0.00028,3,0.297295,2.514736e-01
16,10,0.517666,2.237397e-01,0.765710,0.145040,0.578606,0.00794,3,0.461234,2.237397e-01
17,10,0.609086,2.033153e-01,0.601862,0.234042,0.609726,0.22387,3,0.601509,2.033153e-01
18,10,0.521925,2.244969e-01,0.460989,0.314932,0.533393,0.50000,3,0.520739,2.244969e-01
19,10,0.487238,2.319276e-01,0.417957,0.340162,0.503714,0.60000,3,0.486877,2.319276e-01
20,10,0.372345,2.520507e-01,0.293492,0.409014,0.410946,1.00000,3,0.365975,2.520507e-01
21,11,-0.152864,1.213405e+14,0.338441,0.188850,0.147707,0.00001,3,-0.190200,1.192458e+14
22,11,-0.109998,1.308932e+14,0.309018,0.188061,0.140130,0.00028,3,-0.149731,1.272842e+14
23,11,-0.137783,1.291707e+14,0.312094,0.187324,0.136099,0.00794,3,-0.172332,1.259938e+14


In [70]:
crossdflin = [] 
for m in cv_splitsl:
    samp = errdflin[errdflin[5]==m]
    for speciesc in range(10,12):
        species = speciesc
        sp1 = samp[samp[1]==speciesc]
        for rholamb in laml:
            rho1 = sp1[sp1[0]==rholamb]
            r2 = [] 
            yallbest  = []
            yallact  = [] 
            for iterii in range(m):
                idx = rho1[3].iloc[iterii]
                yact = sst[idx].T[species].values
                ybest = rho1[2].iloc[iterii]
                ybestdf1 = pd.DataFrame(ybest)
                ybestdf1[ybestdf1[0]<1e-3]=0
                slope, intercept,r_value, p_value, std_err = stats.linregress(ybestdf1[0].values,yact)
                yallbest.append(ybestdf1[0].values)
                yallact.append(yact)
            if m == 2:
                ybestagg = [*yallbest[0],*yallbest[1]]
                yactagg = [*yallact[0],*yallact[1]]
            if m == 3:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2]]
            if m == 5:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
            if m == 6:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5]]
            if m == 7:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
            if m == 9:
                ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
                yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
            slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
            #print(slope, intercept,r_value**2)
            #r2df = pd.DataFrame(r2)
            crossdflin.append([speciesc,r2_score(yactagg,ybestagg),mape(yactagg,ybestagg),slope,intercept,r_value**2,rholamb,m])

In [71]:
crossvalin = pd.DataFrame(crossdflin)

In [72]:
crossvalin[crossvalin[7]==3]

,0,1,2,3,4,5,6,7
14,10,0.069211,2.969450e-01,0.793598,0.139459,0.416374,0.00001,3
15,10,0.067844,2.985896e-01,0.792174,0.141241,0.415470,0.00028,3
16,10,0.377839,2.618683e-01,0.760833,0.161596,0.509311,0.00794,3
17,10,0.503325,2.344818e-01,0.491779,0.297467,0.504943,0.22387,3
18,10,0.333494,2.624372e-01,0.279488,0.417150,0.354628,0.50000,3
19,10,0.286504,2.740822e-01,0.233417,0.440533,0.317129,0.60000,3
20,10,0.185513,2.893669e-01,0.143053,0.484646,0.252779,1.00000,3
21,11,-0.770153,2.546014e+14,0.419479,0.296866,0.140474,0.00001,3
22,11,-0.728137,2.473376e+14,0.432625,0.289557,0.149144,0.00028,3
23,11,-0.650939,2.504792e+14,0.412726,0.280284,0.144305,0.00794,3


In [73]:
crossval[crossval[7]==7]

,0,1,2,3,4,5,6,7,8,9
56,10,0.242245,2.773595e-01,0.924605,0.041543,0.532094,0.00001,7,-7.895058e-02,2.786797e-01
57,10,0.256444,2.757332e-01,0.926225,0.041037,0.537604,0.00028,7,-5.924791e-02,2.769583e-01
58,10,0.566869,2.098208e-01,0.913639,0.049799,0.662322,0.00794,7,3.800713e-01,2.098635e-01
59,10,0.691063,1.698599e-01,0.700251,0.173570,0.692055,0.22387,7,5.497420e-01,1.695181e-01
60,10,0.593663,1.980733e-01,0.516460,0.279247,0.610961,0.50000,7,4.605835e-01,1.977869e-01
61,10,0.566234,2.047385e-01,0.472362,0.304004,0.595149,0.60000,7,4.301999e-01,2.043138e-01
62,10,0.431147,2.370302e-01,0.322828,0.392107,0.496429,1.00000,7,2.471179e-01,2.355849e-01
63,11,-0.340264,1.518832e+14,0.069901,0.223126,0.011110,0.00001,7,-1.535065e+06,1.591158e+14
64,11,-0.357361,1.478593e+14,0.070607,0.224851,0.010931,0.00028,7,-1.670821e+06,1.549002e+14
65,11,-0.383484,1.574546e+14,0.044944,0.228897,0.004685,0.00794,7,-1.608846e+06,1.649524e+14


In [ ]:
crossdf = [] 
errdf2 = errdf[:28]
samp = errdf2
m=2
for speciesc in range(2,4):
    species = speciesc
    sp1 = samp[samp[1]==speciesc]
    for rholamb in lamf:
        rho1 = sp1[sp1[0]==rholamb]
        r2 = [] 
        yallbest  = []
        yallact  = [] 
        for iterii in range(2):
            idx = rho1[4].iloc[iterii]
            yact = sst[idx].T[species].values
            ybest = rho1[3].iloc[iterii]
            ybestdf1 = pd.DataFrame(ybest)
            ybestdf1[ybestdf1[0]<1e-3]=0
            yallbest.append(ybestdf1[0].values)
            yallact.append(yact)
            slope, intercept,r_value, p_value, std_err = stats.linregress(ybestdf1[0].values,yact)
            yallbest.append(ybestdf1[0].values)
            yallact.append(yact)
        if m == 2:
            ybestagg = [*yallbest[0],*yallbest[1]]
            yactagg = [*yallact[0],*yallact[1]]
        if m == 3:
            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
            yactagg = [*yallact[0],*yallact[1],*yallact[2]]
        if m == 5:
            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
        if m == 7:
            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
        if m == 9:
            ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6],*yallbest[7],*yallbest[8]]
            yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6],*yallact[7],*yallact[8]]
        slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
        #print(slope, intercept,r_value**2)
        #r2df = pd.DataFrame(r2)
        crossdf.append([speciesc,r2_score(yactagg,ybestagg),slope,intercept,r_value**2,rholamb,m])